PySpark tutorial for beginers

This notebook is a part of my learning journey which I've been documenting from Udacity's Data Scientist Nanodegree program, which
helped me a lot to learn and excel advanced data science stuff such as PySpark. Thank you so much Udacity for providing such quality
content.

Spark is a big data framework which contains libraries for data analysis, machine learning, graph analisis, and streaming live data. Spark
is generally faster than Hadoop. This is because Hadoop writes intermidiate results to disk whereas Spark tries to keep intermediate
results in memory whenever possible.

The Hadoop ecosystem includes a distributed file storage system called HDFS (Hadoop Distributed File System). Spark, on the other
hand, does not include a file storage system. You can use Spark on top of HDFS, but you do not have to. Spark can read in data from
other sources as well such as Amazon S3.

Spark doesn't implement MapReduce, you can write Spark programs that behave in a similar way to the map-reduce pattern.

Limitations of Spark:

    - Spark Streaming's latency is at least 500 milliseconds since it operates on micro-batches of records, instead of processing one
    record at a time. Native streaming tools such as Storm, Apex, of Flink can push down this latency value and might be more suitable
    for low-latency applications. Flink and Apex can be used for batch computation as well, so if you're already using them for stream
    processing, there's no need to ass Spark to your stack of technologies.
    
    - Another limitation of Spark is its selection of machine learning algorithms. Currently, Spark only supports algorithms that scale
    linerly with the input data size. In general, deep learning is not available either, though there are many projects integrate Spark
    with Tensorflow and other deep learning tools.
   
Source: - Spark and PySpark documentation 

In [7]:
import os

for dirname, _, filenames in os.walk(os.getcwd() + "\input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:\00.bigdata\src\log_analysis\log_analysis\input\sparkify_log_small.json
